In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
os.chdir("drive/My Drive/Colab Notebooks")

In [0]:
import numpy as np
import matplotlib.pyplot as plt
# for creating validation set
from sklearn.model_selection import train_test_split
# for evaluating the model
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout, ELU
from torch.optim import Adam, SGD

In [0]:
# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [0]:
X_test = np.load("X_test.npy")
y_test = np.load("y_test.npy")
person_train_valid = np.load("person_train_valid.npy")
X_train_valid = np.load("X_train_valid.npy")
y_train_valid = np.load("y_train_valid.npy")
person_test = np.load("person_test.npy")

In [0]:
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


In [0]:
X_train_valid = X_train_valid[:, np.newaxis, :, :]
X_test = X_test[:, np.newaxis, :, :]
# create labels 
y_train_valid = y_train_valid - 769
y_test = y_test -769

# # # reshape to suit the input of GRU
# X_train_valid = np.swapaxes(X_train_valid, 1, 2)

# split training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train_valid, y_train_valid, test_size = 0.1)

(X_train.shape, y_train.shape), (X_val.shape, y_val.shape)

(((1903, 1, 22, 1000), (1903,)), ((212, 1, 22, 1000), (212,)))

In [0]:
# transform to torch tensor
X_train = torch.from_numpy(X_train)
y_train = torch.from_numpy(y_train)

X_val = torch.from_numpy(X_val)
y_val = torch.from_numpy(y_val)

X_test = torch.from_numpy(X_test)
y_test = torch.from_numpy(y_test)


In [0]:
class GRUNet(nn.Module):
    def __init__(self,input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(GRUNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(input_dim, hidden_dim, n_layers, 
                          dropout = drop_prob, batch_first=True)
    
        self.linear_layers = Sequential(
            Linear(hidden_dim*n_layers, 64),
            nn.BatchNorm1d(64),
            ReLU(inplace=True),
            Linear(64, output_dim)
        )
    
    def forward(self, x, h):
        out, h = self.gru(x, h)
        # print(out.shape)
        # print(h.shape)
        out = self.linear_layers(h.transpose(0, 1).reshape(h.shape[1], -1))
        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device)
        return hidden

# defining the model
model = GRUNet(input_dim = 22, hidden_dim = 64, output_dim = 4, n_layers = 1)
model = model.float()

# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

GRUNet(
  (gru): GRU(22, 64, batch_first=True, dropout=0.2)
  (linear_layers): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=64, out_features=4, bias=True)
  )
)


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


In [0]:
## LSTM ####

class LSTMNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, n_layers, drop_prob=0.2):
        super(LSTMNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.lstm = nn.LSTM(input_dim, hidden_dim, n_layers, batch_first=True, dropout=drop_prob)

        self.linear_layers = Sequential(
            Linear(hidden_dim*n_layers, 64),
            nn.BatchNorm1d(64),
            ReLU(inplace=True),
            Linear(64, output_dim)
        )

    def forward(self, x, h):
        out, h = self.lstm(x, h)
        out = h[1].transpose(0, 1).reshape(h[1].shape[1], -1)
        out = self.linear_layers(out)
        return out, h

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden

# defining the model
model = LSTMNet(input_dim = 22, hidden_dim = 64, output_dim = 4, n_layers = 4)
model = model.float()

# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)


    

LSTMNet(
  (lstm): LSTM(22, 64, num_layers=4, batch_first=True, dropout=0.2)
  (linear_layers): Sequential(
    (0): Linear(in_features=256, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Linear(in_features=64, out_features=4, bias=True)
  )
)


In [0]:
class Net_lstm(nn.Module):
    def __init__(self):
        super(Net_lstm, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=(1,10),  stride=(1,1))
        self.conv1_bn = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(21,1), stride=(1,1))
        self.conv2_bn = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(1,10), stride=(1,1))
        self.conv3_bn = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(1,10), stride=(1,1))
        self.conv4_bn = nn.BatchNorm2d(128)
        self.maxpool = nn.MaxPool2d(kernel_size=(1,4), stride=(1,4), padding=(0,0))
        self.relu = nn.ELU() 
        self.lstm1 = nn.LSTM(128*24, 64, 1, batch_first=True)#, bidirectional=True)
        self.lstm2 = nn.LSTM(64, 64, 1, batch_first=True)#, bidirectional=True)
        self.lstm3 = nn.LSTM(128, 64, 1, batch_first=True)#, bidirectional=True)
        self.lstm4 = nn.LSTM(192, 64, 1, batch_first=True)#, bidirectional=True)
        self.fc = nn.Linear(in_features=64,  out_features=4)
        self.dropout = nn.Dropout(0.2)
        self.softmax = nn.Softmax()
        
        
        
    def forward(self, X):
        # Nx1x22x1000
        x = self.conv1(X)
        x = self.relu(x)
        x = self.conv1_bn(x) 
        x = self.conv2(x)
        x = self.relu(x)
        x = self.conv2_bn(x)
        x = self.maxpool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv3_bn(x)
        x = self.maxpool(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.conv4_bn(x)
        x = self.maxpool(x)
        x = x.permute(0,3,1,2) # important! -- without <50% 128x2x12->12x128x2
        x = x.reshape(-1,1,12*128*2)
        x, _ = self.lstm1(x)
        res1 = x
        x, _ = self.lstm2(x)
        res2 = x
        x = torch.cat((x, res1), dim=2)
        x, _ = self.lstm3(x)
        x = torch.cat((x, res1, res2), dim=2)
        x,_ = self.lstm4(x)
        x = x.squeeze()
        x = self.fc(x)
        return x



In [0]:
# defining the model
model = Net_lstm()
model = model.float()

# defining the loss function
criterion = CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

print(model)

Net_lstm(
  (conv1): Conv2d(1, 16, kernel_size=(1, 10), stride=(1, 1))
  (conv1_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(16, 32, kernel_size=(21, 1), stride=(1, 1))
  (conv2_bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(32, 64, kernel_size=(1, 10), stride=(1, 1))
  (conv3_bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(64, 128, kernel_size=(1, 10), stride=(1, 1))
  (conv4_bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (maxpool): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=(0, 0), dilation=1, ceil_mode=False)
  (relu): ELU(alpha=1.0)
  (lstm1): LSTM(3072, 64, batch_first=True)
  (lstm2): LSTM(64, 64, batch_first=True)
  (lstm3): LSTM(128, 64, batch_first=True)
  (lstm4): LSTM(192, 64, batch_first=True)
  (fc): Linear(in_features=64, out_features=4, bias=True)
  (drop

In [0]:
# process the data in batches for training
batch_size = 50
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

optimizer = Adam(model.parameters(), lr=1e-4)
n_epochs = 50
n_batches = len(y_train)//batch_size
count = 0

for epoch in range(n_epochs):
    print_every = n_batches // 10
    total_train_loss = 0
    for i,data in enumerate(train_loader, 0):
        count += 1

        # extract data in this batch
        inputs, labels = data

        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        #Set the parameter gradients to zero
        optimizer.zero_grad()
        
        #Forward pass, backward pass, optimize
        out= model(inputs.float())
        loss_train = criterion(out, labels.long())
        loss_train.backward()
        optimizer.step()

        #train loss
        total_train_loss += loss_train

        #Print every 100 iteration of an epoch
        if count % 50 == 0:
            print("Epoch {}, Step: {}/{}, \t train_loss: {:.2f} ".format(
                    epoch+1, count, len(train_loader), total_train_loss))
            
    if (epoch+1) % 5 == 0:    
        ## At the end of the epoch, do a pass on the validation set
        out = model(X_val.cuda().float())
        loss_val = criterion(out, y_val.to(device).long())
        print("Epoch {}, \t validation loss = {:.2f}".format(epoch+1, loss_val))

Epoch 2, Step: 50/38, 	 train_loss: 0.71 
Epoch 3, Step: 100/38, 	 train_loss: 1.29 
Epoch 4, Step: 150/38, 	 train_loss: 1.34 
Epoch 5, 	 validation loss = 2.38
Epoch 6, Step: 200/38, 	 train_loss: 0.29 
Epoch 7, Step: 250/38, 	 train_loss: 1.35 
Epoch 8, Step: 300/38, 	 train_loss: 2.55 
Epoch 10, Step: 350/38, 	 train_loss: 1.08 
Epoch 10, 	 validation loss = 2.49
Epoch 11, Step: 400/38, 	 train_loss: 1.33 
Epoch 12, Step: 450/38, 	 train_loss: 0.90 
Epoch 14, Step: 500/38, 	 train_loss: 0.11 
Epoch 15, Step: 550/38, 	 train_loss: 0.13 
Epoch 15, 	 validation loss = 2.52
Epoch 16, Step: 600/38, 	 train_loss: 0.20 
Epoch 18, Step: 650/38, 	 train_loss: 0.01 
Epoch 19, Step: 700/38, 	 train_loss: 0.02 
Epoch 20, Step: 750/38, 	 train_loss: 0.03 
Epoch 20, 	 validation loss = 2.70
Epoch 22, Step: 800/38, 	 train_loss: 0.00 
Epoch 23, Step: 850/38, 	 train_loss: 0.01 
Epoch 24, Step: 900/38, 	 train_loss: 0.02 
Epoch 25, Step: 950/38, 	 train_loss: 0.02 
Epoch 25, 	 validation loss = 2.

In [0]:
## validation accuracy
with torch.no_grad():
    output = model(X_val.cuda().float())
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(y_val.cpu(), predictions)

0.5235849056603774

In [0]:
## test accuracy
with torch.no_grad():
    output = model(X_test.to(device).cuda().float())
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(y_test.cpu(), predictions)

0.5304740406320542

In [0]:
## training procedure for LSTM and GRU (ONLY)
# process the data in batches for training
batch_size = 100
train_data = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_data, shuffle=True, batch_size=batch_size, drop_last=True)

optimizer = Adam(model.parameters(), lr=1e-2)
n_epochs = 10
n_batches = len(y_train)//batch_size

for epoch in range(n_epochs):
    running_loss = 0
    print_every = n_batches // 10
    total_train_loss = 0
    h = model.init_hidden(batch_size)
    for i,data in enumerate(train_loader, 0):
        # extract data in this batch
        inputs, labels = data

        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()
        
        #Set the parameter gradients to zero
        optimizer.zero_grad()
        
        #Forward pass, backward pass, optimize
        h = tuple([e.data for e in h])
        out, h = model(inputs.float(), h)
        loss_train = criterion(out, labels.long())
        loss_train.backward()
        optimizer.step()

        #Print statistics
        running_loss += loss_train
        total_train_loss += loss_train
        
        #Print every 10th batch of an epoch
        if (i + 1) % (print_every + 1) == 0:
            print("Epoch {}, {:d}% \t train_loss: {:.2f} ".format(
                    epoch+1, int(100 * (i+1) / n_batches), running_loss / print_every))
            #Reset running loss and time
            running_loss = 0.0
        
    # #At the end of the epoch, do a pass on the validation set
    h = model.init_hidden(len(y_val))
    out, h = model(X_val.cuda().float(), h)
    loss_val = criterion(out, y_val.to(device).long())

    print("Validation loss = {:.2f}".format(loss_val))

Epoch 1, 10% 	 train_loss: 2.96 
Epoch 1, 21% 	 train_loss: 2.93 
Epoch 1, 31% 	 train_loss: 2.92 
Epoch 1, 42% 	 train_loss: 2.86 
Epoch 1, 52% 	 train_loss: 2.83 
Epoch 1, 63% 	 train_loss: 2.83 
Epoch 1, 73% 	 train_loss: 2.83 
Epoch 1, 84% 	 train_loss: 2.86 
Epoch 1, 94% 	 train_loss: 2.78 
Validation loss = 1.38
Epoch 2, 10% 	 train_loss: 2.77 
Epoch 2, 21% 	 train_loss: 2.81 
Epoch 2, 31% 	 train_loss: 2.81 
Epoch 2, 42% 	 train_loss: 2.78 
Epoch 2, 52% 	 train_loss: 2.73 
Epoch 2, 63% 	 train_loss: 2.77 
Epoch 2, 73% 	 train_loss: 2.84 
Epoch 2, 84% 	 train_loss: 2.84 
Epoch 2, 94% 	 train_loss: 2.78 
Validation loss = 1.39
Epoch 3, 10% 	 train_loss: 2.76 
Epoch 3, 21% 	 train_loss: 2.78 
Epoch 3, 31% 	 train_loss: 2.80 
Epoch 3, 42% 	 train_loss: 2.73 
Epoch 3, 52% 	 train_loss: 2.75 
Epoch 3, 63% 	 train_loss: 2.76 
Epoch 3, 73% 	 train_loss: 2.74 
Epoch 3, 84% 	 train_loss: 2.76 
Epoch 3, 94% 	 train_loss: 2.73 
Validation loss = 1.41
Epoch 4, 10% 	 train_loss: 2.75 
Epoch 4

In [0]:
h[1].shape

torch.Size([4, 212, 64])

In [0]:
## validation accuracy
h = model.init_hidden(len(y_val))
with torch.no_grad():
    output, h = model(X_val.cuda().float(), h)
softmax = torch.exp(output).cpu()
prob = list(softmax.numpy())
predictions = np.argmax(prob, axis=1)

# accuracy on validation set
accuracy_score(y_val.cpu(), predictions)

AttributeError: ignored